In [ ]:
# Find top packers & movers — LinkUp and Perplexity examples

This notebook contains three examples to help you find packers & movers for a given address:

1. LinkUp API — template (replace endpoint/keys with LinkUp docs)
2. Perplexity API — template (replace endpoint/keys with Perplexity docs)
3. OpenStreetMap + Overpass — working example (no API key required)

Notes / assumptions:
- I don't have the exact public endpoints for LinkUp or Perplexity in this repo, so the first two cells are templates you can adapt to the official API docs (you'll need to fill in the correct endpoint and response-field names).
- The third cell is a fully working approach using public OpenStreetMap services (Nominatim + Overpass) and will work as-is for most addresses. It's a good fallback if you don't have API access.

Replace the placeholder API key environment variables (`LINKUP_API_KEY`, `PERPLEXITY_API_KEY`) before running the template cells.

Run each cell independently.

In [ ]:
# LinkUp API template: replace endpoint and parsing according to LinkUp API docs
# Set LINKUP_API_KEY in environment or replace below
import os, requests

API_KEY = os.getenv('LINKUP_API_KEY', 'YOUR_LINKUP_API_KEY')
ADDRESS = "825 Menlo Ave, Menlo Park, CA 94002"

headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

# NOTE: Replace the URL and payload with the real LinkUp search endpoint and parameters
url = 'https://api.linkup.example/v1/search_businesses'
payload = {
    'query': 'movers OR moving company OR packers',
    'address': ADDRESS,
    'limit': 10
}

resp = requests.post(url, json=payload, headers=headers)
print('Status:', resp.status_code)
try:
    data = resp.json()
    # Adapt parsing to real response structure
    results = data.get('businesses', [])
    for i, r in enumerate(results[:5], 1):
        print(i, r.get('name'), '-', r.get('address'))
except Exception as e:
    print('Failed to parse JSON:', e)


# If you have a real LinkUp spec, replace the URL/payload/fields above.

In [ ]:
# Perplexity API template: replace endpoint and parsing according to Perplexity docs
import os, requests

API_KEY = os.getenv('PERPLEXITY_API_KEY', 'YOUR_PERPLEXITY_API_KEY')
ADDRESS = "825 Menlo Ave, Menlo Park, CA 94002"

headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

# NOTE: Replace this with the correct Perplexity API search/answer endpoint and parameters
url = 'https://api.perplexity.ai/search'
payload = {
    'query': f"Top moving companies near \"{ADDRESS}\"",
    'limit': 10
}

resp = requests.post(url, json=payload, headers=headers)
print('Status:', resp.status_code)
try:
    data = resp.json()
    # Adapt parsing to real response structure
    answers = data.get('answers', [])
    for i, a in enumerate(answers[:5], 1):
        print(i, a.get('title') or a.get('text')[:100])
except Exception as e:
    print('Failed to parse JSON:', e)


# OpenStreetMap + Overpass working example (no API key required)
# This cell geocodes the address with Nominatim and queries Overpass for moving companies ("moving" or "movings" tags are uncommon; we search for "moving" in name/type)
import requests

ADDRESS = "825 Menlo Ave, Menlo Park, CA 94002"

# 1) Geocode with Nominatim
geocode_url = 'https://nominatim.openstreetmap.org/search'
params = {
    'q': ADDRESS,
    'format': 'json',
    'limit': 1
}
res = requests.get(geocode_url, params=params, headers={'User-Agent': 'movescout-example/1.0'})
res.raise_for_status()
geo = res.json()
if not geo:
    raise SystemExit('Address not found')
lat = float(geo[0]['lat'])
lon = float(geo[0]['lon'])
print('Geocoded to:', lat, lon)

# 2) Query Overpass for businesses with keywords near the point
overpass_url = 'https://overpass-api.de/api/interpreter'
# Search within 5km radius; tune 'around' as needed
overpass_query = f'''
[out:json][timeout:25];
(
  node(around:5000,{lat},{lon})["name"~"[Mm]over|[Mm]oving|[Pp]ackers|[Pp]ackers|[Mm]overs"]; 
  way(around:5000,{lat},{lon})["name"~"[Mm]over|[Mm]oving|[Pp]ackers|[Pp]ackers|[Mm]overs"];
  rel(around:5000,{lat},{lon})["name"~"[Mm]over|[Mm]oving|[Pp]ackers|[Pp]ackers|[Mm]overs"];
);
out center;'
'''

resp = requests.post(overpass_url, data={'data': overpass_query}, headers={'User-Agent': 'movescout-example/1.0'})
resp.raise_for_status()
results = resp.json().get('elements', [])

# Format and print top results by distance
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    # return distance in km
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    km = 6371 * c
    return km

parsed = []
for el in results:
    name = el.get('tags', {}).get('name')
    addr = []
    for k in ['addr:housenumber','addr:street','addr:city','addr:postcode']:
        if el.get('tags', {}).get(k):
            addr.append(el.get('tags', {}).get(k))
    distance = haversine(lat, lon, el.get('lat') or el.get('center',{}).get('lat'), el.get('lon') or el.get('center',{}).get('lon'))
    parsed.append({'name':name,'address':', '.join(addr),'distance_km':distance,'raw':el})

parsed_sorted = sorted([p for p in parsed if p['name']], key=lambda x: x['distance_km'])
for i,p in enumerate(parsed_sorted[:10],1):
    print(i, p['name'], f"({p['distance_km']:.2f} km)", '-', p['address'])
